<a href="https://colab.research.google.com/github/BartoszJanJerzy/M-L-Bootcamp-cz.1/blob/master/01_preprocessing_intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
import numpy as np
import pandas as pd
import sklearn

##Generowanie danych

In [36]:
data = {
    'size': ['XL','L','M','L','M'],
    'color':['red','green','blue','green','red'],
    'gender':['female','male','male','female','female'],
    'price':[199.0,89.0,99.0,129.0,79.0],
    'weight':[500,450,300,380,410],
    'bought':['yes','no','yes','no','yes']
}

df_raw = pd.DataFrame(data)
df_raw

,size,color,gender,price,weight,bought
0,XL,red,female,199.0,500,yes
1,L,green,male,89.0,450,no
2,M,blue,male,99.0,300,yes
3,L,green,female,129.0,380,no
4,M,red,female,79.0,410,yes


In [37]:
# KOPIA DANYCH
df = df_raw.copy()

In [38]:
# INFORMACJE O DANYCH
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   size    5 non-null      object 
 1   color   5 non-null      object 
 2   gender  5 non-null      object 
 3   price   5 non-null      float64
 4   weight  5 non-null      int64  
 5   bought  5 non-null      object 
dtypes: float64(1), int64(1), object(4)
memory usage: 368.0+ bytes


##Zmiana typów danych i wstępna eksploracja

In [39]:
# ZMIANA TYPÓW NA 'CATEGORY': lepiej się sprawdza w wizualizacjach
for col in ['size','color','gender','bought']:
  df[col] = df[col].astype('category')

# ZMIANA TYPU Z 'INT64' NA ;FLOAT': POTRZEBNE DO STANDARYZACJI
df['weight'] = df['weight'].astype('float')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype   
---  ------  --------------  -----   
 0   size    5 non-null      category
 1   color   5 non-null      category
 2   gender  5 non-null      category
 3   price   5 non-null      float64 
 4   weight  5 non-null      float64 
 5   bought  5 non-null      category
dtypes: category(4), float64(2)
memory usage: 628.0 bytes


In [40]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
price,5.0,119.0,48.476799,79.0,89.0,99.0,129.0,199.0
weight,5.0,408.0,75.299402,300.0,380.0,410.0,450.0,500.0


In [41]:
df.describe(include=['category']).T

,count,unique,top,freq
size,5,3,M,2
color,5,3,red,2
gender,5,2,female,3
bought,5,2,yes,3


##LabelEncoder - mapowanie zmiennych

In [42]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder() #tworzenie instancji klasy
le.fit(df['bought']) # dopasowanie encodera do zmiennej
le.transform(df['bought']) # transforamcja do wartości numerycznych: zwraca tablice array
# można szybciej od razu le.fit_transform(df['bought'])

array([1, 0, 1, 0, 1])

In [43]:
# JAKIE MMAY KLASY W TEJ ZMIENNEJ?
le.classes_

array(['no', 'yes'], dtype=object)

In [44]:
# zamiana zmiennej w df
df['bought'] = le.fit_transform(df['bought'])
df

# powrót do pierwotnego stanu: df['bought'] = le.inverse_transform(df['bought])

,size,color,gender,price,weight,bought
0,XL,red,female,199.0,500.0,1
1,L,green,male,89.0,450.0,0
2,M,blue,male,99.0,300.0,1
3,L,green,female,129.0,380.0,0
4,M,red,female,79.0,410.0,1


## OneHotEncoder - modyfikacja predyktorów nominalnych

In [45]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse=False) #gdyby dać True to zwróciłby macierz rzadką: tylko info gdzie są 1, bez kodowania 0: oszczędność pamięci
encoder.fit(df[['size']])
encoder.transform(df[['size']])

array([[0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.]])

In [46]:
# obecnie jest za dużo kolumn: zob wiersz 1
  #   są 0 0 1 -> XL
  #   równie dobrze może być tylko 0 0 -> XL
  #   jedna z kolumn jest zbędna
# daltego jeszcze raz ale z dodatkowym parametrem

encoder = OneHotEncoder(sparse=False, drop='first') 
encoder.fit(df[['size']])
encoder.transform(df[['size']])

array([[0., 1.],
       [0., 0.],
       [1., 0.],
       [0., 0.],
       [1., 0.]])

In [47]:
# Jak jest teraz bez pierwszej kolumny?
#     0 1 -> XL
#     0 0 -> L
#     1 0 -> M
encoder.categories_

[array(['L', 'M', 'XL'], dtype=object)]

## To samo tylko za pomoca bilbioteki PANDAS

In [48]:
df = pd.get_dummies(data=df, drop_first=True)
df

,price,weight,bought,size_M,size_XL,color_green,color_red,gender_male
0,199.0,500.0,1,0,1,0,1,0
1,89.0,450.0,0,0,0,1,0,1
2,99.0,300.0,1,1,0,0,0,1
3,129.0,380.0,0,0,0,1,0,0
4,79.0,410.0,1,1,0,0,1,0


## Standaryzacja zmiennych ilościowych
z = (x-M) / SD

różnice w wynikach po uzyciu różnych bibliotek: pandas uzywa std nieobciążone, sklearn obciążone

In [49]:
from sklearn.preprocessing import scale # sposób na szypko.. ale bedzie potem lepszy

std_price = scale(df['price'])
std_weight = scale(df['weight'])
print(std_price)
print(std_weight)

[ 1.84506242 -0.69189841 -0.4612656   0.2306328  -0.92253121]
[ 1.3660019   0.62360956 -1.60356745 -0.41573971  0.02969569]


In [50]:
from sklearn.preprocessing import StandardScaler #ten jest lepszy bo zachowuej info o źródłowych zmiennych

scaler = StandardScaler()
std_price = scaler.fit_transform(df[['price']])
std_weight = scaler.fit_transform(df[['weight']])
print(std_price)
print()
print(std_weight)

[[ 1.84506242]
 [-0.69189841]
 [-0.4612656 ]
 [ 0.2306328 ]
 [-0.92253121]]

[[ 1.3660019 ]
 [ 0.62360956]
 [-1.60356745]
 [-0.41573971]
 [ 0.02969569]]


In [51]:
df['price'] = std_price
df['weight'] = std_weight
df

,price,weight,bought,size_M,size_XL,color_green,color_red,gender_male
0,1.845062,1.366002,1,0,1,0,1,0
1,-0.691898,0.623610,0,0,0,1,0,1
2,-0.461266,-1.603567,1,1,0,0,0,1
3,0.230633,-0.415740,0,0,0,1,0,0
4,-0.922531,0.029696,1,1,0,0,1,0


In [52]:
# a jak użyć StandardScaler.. ale szypko?
scaler = StandardScaler()
df[['price','weight']] = scaler.fit_transform(df[['price','weight']])
df

,price,weight,bought,size_M,size_XL,color_green,color_red,gender_male
0,1.845062,1.366002,1,0,1,0,1,0
1,-0.691898,0.623610,0,0,0,1,0,1
2,-0.461266,-1.603567,1,1,0,0,0,1
3,0.230633,-0.415740,0,0,0,1,0,0
4,-0.922531,0.029696,1,1,0,0,1,0
